<a href="https://colab.research.google.com/github/abhishek1397/Drone_Detection/blob/master/Train_yolov3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [0]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sat Apr  4 13:13:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive \yolov3
!ls /mydrive/yolov3/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: target 'yolov3' is not a directory
images.zip


**1) Clone the Darknet**



In [0]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 13033 (delta 2), reused 5 (delta 2), pack-reused 13025
Receiving objects: 100% (13033/13033), 11.77 MiB | 12.25 MiB/s, done.
Resolving deltas: 100% (8902/8902), done.


**2) Compile Darknet using Nvidia GPU**


In [0]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warnin

**3) Configure Darknet network for training YOLO V3**

In [0]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [0]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [0]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [0]:
!echo "Drone" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

**4) Download darknet53 model**

In [0]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-04-04 13:16:04--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   461KB/s    in 5m 25s  

2020-04-04 13:21:31 (488 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**5) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be on the main folder

In [0]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Streaming output truncated to the last 5000 lines.
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990670.txt  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990673.jpg  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990673.txt  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990676.jpg  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990676.txt  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990679.jpg  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990679.txt  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990682.jpg  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990682.txt  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990685.jpg  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990685.txt  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990688.jpg  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438

In [0]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/circa-january-6-2016-dji-260nw-359234735.jpg', 'data/obj/istanbul-turkey-july-3-2018-260nw-1131337241.jpg', 'data/obj/los-angeles-ca-usa-june-260nw-1121431799.jpg', 'data/obj/istanbul-turkey-july-3-2018-260nw-1130725391.jpg', 'data/obj/pecs-baranya-hungary-february-19-260nw-1029294976.jpg', 'data/obj/pilsen-czech-republic-june-28-260nw-291448274.jpg', 'data/obj/moscow-russia01-apr-2017-dji-260nw-615678164.jpg', 'data/obj/dji-mavic-pro-drone-rigalatvia-260nw-542803249.jpg', 'data/obj/st-petersburg-russia-june-4-260nw-455414860.jpg', 'data/obj/pilsen-czech-republic-january-19-260nw-560273257.jpg', 'data/obj/udonthani-thailand-december-11-2015-260nw-371324662.jpg', 'data/obj/astana-kazakhstan-december-18-2017-260nw-777449344.jpg', 'data/obj/dji-mavic-pro-drone-riga-260nw-533974813.jpg', 'data/obj/pilsen-czech-republic-july-26-260nw-299942411.jpg', 'data/obj/italyoct30-dji-phantom-quadcopter-flies-260nw-239734924.jpg', 'data/obj/skadovsk-ukraine-june-15-2018-260nw-1121976152.jpg

In [0]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [0]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show